In [1]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque

offene Probleme / mögliche Verbesserungen:
    - in einem Endzustand nicht mehr q(a',s') berechnen
    - Minibatch mit TF erzeugen
    - store old network weights and use them in loss function

In [2]:
# build graph
obs_dim = 4
n_acts = 2
gamma = 0.9

obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None, 1), dtype=tf.int32)
rew_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32)
new_obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)

# make core of state-action-value function network
mlp_action_val = tf.keras.models.Sequential()
mlp_action_val.add(tf.keras.layers.Dense(10, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(10, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(n_acts))

# define state action values
old_state_action_values = mlp_action_val(obs_ph)
new_state_action_values = mlp_action_val(new_obs_ph)

# select action
greedy_action = tf.math.argmax(old_state_action_values)

# define loss function
y = rew_ph + gamma * np.max(new_state_action_values)
y_tensor = tf.identity(y)
action_masks = tf.one_hot(act_ph, n_acts)
old_selected_action_values = tf.reduce_sum(action_masks * old_state_action_values, axis=1)
action_value_loss = tf.reduce_mean((y_tensor - old_selected_action_values)**2)

# define optimizer
optimizer_action_value = tf.train.AdamOptimizer(0.0001)
train_action_value = optimizer_action_value.minimize(action_value_loss)


In [3]:
def dqn(sess, env, experience_replay_buffer, n_episodes):
    for i in range(n_episodes):
        done = False
        state = env.reset()
        cnt = 0
        while not done:
            action = select_action(sess, state)
            new_state, reward, done, _ = env.step(action)
            experience_replay_buffer.append((state, action, reward, new_state))
            state = new_state
            update_q_function(experience_replay_buffer)
            cnt += 1
        print(i, cnt)
        
        
def update_q_function(experience_replay_buffer):
    experience_batch = sample_experience_batch(experience_replay_buffer)
    states, actions, rewards, new_states = extract_data(experience_batch)
    loss = gradient_step(states, actions, rewards, new_states)

def gradient_step(states, actions, rewards, new_states):
    _, loss = sess.run([train_action_value, action_value_loss], feed_dict={
                                obs_ph: np.array(states).reshape(-1, obs_dim),
                                act_ph: np.array(actions).reshape(-1,1),
                                rew_ph: np.array(rewards).reshape(-1,1),
                                new_obs_ph: np.array(new_states).reshape(-1,obs_dim)
                             })
    return loss
    
def extract_data(experience_batch):
    states = [x[0] for x in experience_batch]
    actions = [x[1] for x in experience_batch]
    rewards = [x[2] for x in experience_batch]
    new_states = [x[3] for x in experience_batch]
    return states, actions, rewards, new_states
    
def sample_experience_batch(experience_replay_buffer, batch_size=100):
    indices = np.random.choice(np.arange(len(experience_replay_buffer)), batch_size)
    tmp = np.array(experience_replay_buffer)
    sample_batch = tmp[indices, :]
#     sample_batch = np.random.choice(experience_replay_buffer,batch_size, replace = False)
    return sample_batch

def select_action(sess, state):
    print(sess.run(old_state_action_values, {obs_ph: state.reshape(1,-1)}))
    action = 0
    if np.random.rand() < 0.1:
        action = np.random.randint(2)
    else:
        action = sess.run(greedy_action, {obs_ph: state.reshape(1,-1)})[0]
    return action

In [4]:
# main
exp_replay_size = 5000
experience_replay_buffer = deque([], exp_replay_size)
env = gym.make('CartPole-v0')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

dqn(sess, env, experience_replay_buffer, 1000)

[[ 0.00245213 -0.00102579]]
[[-0.6048982  -0.03489596]]
[[-0.00898822 -0.00102801]]
[[0.5879381  0.03219982]]
[[1.0671582  0.08317722]]
[[1.3985177  0.14693616]]
[[1.6118941  0.21083874]]
[[1.7460377  0.26798567]]
[[1.8305894  0.31653166]]
[[1.8854171  0.35678697]]
[[1.9232144  0.38983795]]
[[1.9518894  0.41695186]]
0 12
[[-0.0329962  -0.00663375]]
[[0.55599916 0.02455158]]
[[1.0318407 0.0732414]]
[[1.3631343  0.13579528]]
[[1.5776209  0.19965942]]
[[1.7129523 0.2573998]]
[[1.7984223  0.30671537]]
[[1.8538605  0.34769523]]
[[1.8272506 0.324812 ]]
[[1.878658   0.36381692]]
1 10
[[-0.087456   -0.00147528]]
[[0.52595717 0.0293291 ]]
[[1.0263165  0.08084802]]
[[1.3691695  0.14646696]]
[[1.584478   0.21135508]]
[[1.7157393 0.268372 ]]
[[1.7963088  0.31612447]]
[[1.8477757  0.35535198]]
2 8
[[ 0.01515056 -0.00093024]]
[[0.60593337 0.03080727]]
[[1.0702083  0.08360822]]
[[1.3831737  0.14878464]]
[[1.579478   0.21259281]]
[[1.6999483  0.26860937]]
[[1.7746314  0.31552592]]
[[1.8229766 0.354001

[[0.75860935 0.08930148]]
[[0.8302689 0.1299235]]
[[0.88242364 0.16682625]]
[[0.9228618  0.19893977]]
[[0.9566144  0.22619715]]
34 12
[[-0.2811248  -0.02234822]]
[[ 0.02648361 -0.02431321]]
[[-0.30207714 -0.02061334]]
[[ 0.00609336 -0.0229489 ]]
[[ 0.2793358  -0.01607199]]
[[0.4880658  0.00776046]]
[[0.6350437  0.04391325]]
[[0.73615927 0.0847568 ]]
[[0.8069375 0.1248087]]
[[0.8588398  0.16124572]]
[[0.89945674 0.19297393]]
[[0.9336559 0.219957 ]]
35 12
[[-0.25385895 -0.02579131]]
[[ 0.05820408 -0.02846308]]
[[ 0.32611898 -0.01689102]]
[[0.5247822  0.01285782]]
[[0.6619246  0.05324514]]
[[0.7554577  0.09607606]]
[[0.8211694  0.13642667]]
[[0.8700689  0.17211702]]
[[0.90921503 0.2026245 ]]
36 9
[[-0.26505476 -0.02777559]]
[[ 0.03779931 -0.03110453]]
[[ 0.29935068 -0.02163012]]
[[0.4951384  0.00566403]]
[[0.63168675 0.04424028]]
[[0.725734   0.08613062]]
[[0.7923037 0.1262012]]
[[0.8420322  0.16200817]]
[[0.88175875 0.19279845]]
[[0.91587836 0.2187659 ]]
37 10
[[-0.2568031  -0.03098299]]

[[ 0.26318687 -0.00162203]]
[[0.33951518 0.03487225]]
[[0.3999224  0.07044154]]
[[0.45045865 0.10257274]]
69 10
[[-0.3911871 -0.0327932]]
[[-0.2049654  -0.04521145]]
[[-0.03421709 -0.05023104]]
[[ 0.10094456 -0.03947797]]
[[ 0.20171535 -0.01526536]]
[[0.277787   0.01605586]]
[[0.3382417  0.04916882]]
[[0.38912985 0.08084057]]
[[0.43389314 0.10945958]]
[[0.47465244 0.13449486]]
70 10
[[-0.38336787 -0.03286212]]
[[-0.1919653  -0.04642267]]
[[-0.01691885 -0.049592  ]]
[[ 0.12030894 -0.03525278]]
[[ 0.22179432 -0.00765099]]
[[0.2981517  0.02595714]]
[[0.3587776  0.06016196]]
[[0.40971902 0.09198707]]
[[0.45448375 0.12015058]]
71 9
[[-0.4186898  -0.03906746]]
[[-0.23554695 -0.05098052]]
[[-0.4253392  -0.03754971]]
[[-0.24256298 -0.0495597 ]]
[[-0.06919362 -0.05586331]]
[[ 0.07108364 -0.04650043]]
[[ 0.17655244 -0.02274515]]
[[0.25616583 0.00899522]]
[[0.3191175 0.0429508]]
[[0.3717535  0.07555804]]
[[0.41783178 0.10504454]]
[[0.40631196 0.08951818]]
72 12
[[-0.4175379 -0.0385832]]
[[-0.2362

[[ 0.14196336 -0.00266324]]
[[0.21307607 0.03396069]]
[[0.17248508 0.00782873]]
[[0.24436763 0.04600804]]
105 10
[[-0.43101612 -0.03901932]]
[[-0.5658089  -0.03472101]]
[[-0.6550431 -0.0463405]]
[[-0.57479733 -0.03450311]]
[[-0.45244196 -0.03492838]]
[[-0.30581522 -0.04458464]]
[[-0.16390023 -0.05149001]]
[[-0.0460922  -0.04563437]]
[[ 0.0454301  -0.02634166]]
[[-0.0427295  -0.04300939]]
[[ 0.05353343 -0.02293466]]
[[-0.03313062 -0.0405639 ]]
[[ 0.06662285 -0.01875798]]
[[0.14403383 0.01159265]]
[[0.2071319 0.0445858]]
[[0.26141888 0.07652736]]
[[0.30999708 0.10554998]]
106 17
[[-0.3920163  -0.03636421]]
[[-0.23250403 -0.0480755 ]]
[[-0.08640239 -0.04819702]]
[[ 0.02960001 -0.03151102]]
[[ 0.11826718 -0.00286474]]
[[0.18839616 0.03080272]]
[[0.24704671 0.06452613]]
[[0.29843912 0.09563681]]
107 8
[[-0.4395768  -0.03563023]]
[[-0.29110438 -0.04561709]]
[[-0.14936106 -0.05130895]]
[[-0.03289153 -0.04343119]]
[[ 0.0572649  -0.02231401]]
[[0.12828627 0.00677165]]
[[0.1873026  0.03865724]]


[[-0.3097612  -0.04954781]]
[[-0.16925934 -0.05508791]]
[[-0.05234063 -0.04744839]]
[[ 0.03878919 -0.02629047]]
[[0.11073021 0.00318953]]
[[ 0.0518722  -0.02142257]]
[[0.1276331  0.00962688]]
[[0.18996389 0.04324844]]
[[0.24397635 0.07571851]]
[[0.29246095 0.10514393]]
[[0.33698153 0.13087353]]
138 12
[[-0.4183032  -0.03186757]]
[[-0.26996994 -0.04239717]]
[[-0.1304349  -0.04622226]]
[[-0.01760199 -0.03532946]]
[[ 0.06924304 -0.01160974]]
[[0.13807508 0.01902591]]
[[0.19593592 0.05146121]]
[[0.24707112 0.08251829]]
[[0.29373357 0.11063915]]
[[0.33700556 0.13523233]]
139 10
[[-0.41360334 -0.03885005]]
[[-0.2575268  -0.05038502]]
[[-0.11064704 -0.05114351]]
[[ 0.00777866 -0.03485687]]
[[-0.09236935 -0.05075178]]
[[-0.21950008 -0.05559293]]
[[-0.0708087  -0.05065075]]
[[ 0.04700098 -0.02774818]]
[[0.13719557 0.00580707]]
[[0.20859332 0.04242083]]
[[0.26814356 0.07739842]]
140 11
[[-0.41857445 -0.04775465]]
[[-0.2666678  -0.05763966]]
[[-0.125073   -0.05857503]]
[[-0.01036446 -0.04373001]]

...

[[-0.05754303 -0.0390456 ]]
[[ 0.00673095 -0.03332429]]
[[ 0.05292019 -0.01427275]]
[[ 0.01614545 -0.030577  ]]
[[ 0.06535466 -0.0093645 ]]
[[0.10507558 0.01726487]]
[[0.14094187 0.04448137]]
[[0.17541593 0.06960346]]
840 10
[[-0.28001052  0.00242137]]
[[-0.17374273 -0.02428206]]
[[-0.08158665 -0.0394531 ]]
[[-0.01347088 -0.03670412]]
[[ 0.03532197 -0.02013084]]
[[0.07360607 0.00356552]]
[[0.10791387 0.02939032]]
[[0.1414279  0.05442572]]
[[0.17491356 0.0773249 ]]
[[0.2082455  0.09747533]]
841 10
[[-0.27341872 -0.00140035]]
[[-0.1619394  -0.02919982]]
[[-0.06457132 -0.04279878]]
[[ 0.00766349 -0.03668113]]
[[ 0.06025078 -0.01649111]]
[[0.10211317 0.01011453]]
[[0.13949305 0.03788541]]
[[0.17501427 0.06393887]]
[[0.20955703 0.08713238]]
842 9
[[-0.2726432   0.01288713]]
[[-0.1657665  -0.01382142]]
[[-0.07278982 -0.03138322]]
[[-0.00524854 -0.03189986]]
[[ 0.04181024 -0.018092  ]]
[[0.0776118 0.0038587]]
[[ 0.05086567 -0.01385987]]
[[ 0.0084653  -0.02779178]]
[[ 0.06151782 -0.00955668]]


[[ 0.02755255 -0.02551928]]
[[ 0.06354521 -0.00383327]]
[[0.09522223 0.02106956]]
[[0.12595423 0.04588411]]
[[0.11485653 0.02948763]]
[[0.14917304 0.05494252]]
[[0.18278754 0.07793599]]
869 11
[[-0.27658218 -0.00450769]]
[[-0.1695995  -0.03231061]]
[[-0.07622932 -0.046841  ]]
[[-0.00741354 -0.04218296]]
[[ 0.04244363 -0.02329451]]
[[0.08202183 0.002474  ]]
[[ 0.0561096  -0.01804945]]
[[0.09931225 0.0092765 ]]
[[0.13794371 0.03739981]]
[[0.17456076 0.06347304]]
870 10
[[-0.27768633  0.00493745]]
[[-0.3671322   0.02796094]]
[[-0.27244365  0.00229107]]
[[-0.16135706 -0.02703261]]
[[-0.06351233 -0.04230896]]
[[ 0.0088404  -0.03748311]]
[[ 0.06037008 -0.01815019]]
[[0.10035269 0.00783746]]
[[ 0.07540356 -0.01244551]]
[[0.1184019  0.01534402]]
[[0.1560885  0.04338995]]
871 11
[[-0.24731478 -0.00129115]]
[[-0.13667089 -0.02957153]]
[[-0.0436351  -0.04159996]]
[[ 0.02309317 -0.03374539]]
[[ 0.07056686 -0.01268163]]
[[0.10787942 0.01386851]]
[[ 0.08510122 -0.00648604]]
[[0.12544085 0.02182554]]

[[-0.24988683  0.00109234]]
[[-0.14520052 -0.02714882]]
[[-0.05656478 -0.04132423]]
[[ 0.00742953 -0.03630674]]
[[-0.05156944 -0.0411015 ]]
[[ 0.01472595 -0.0344448 ]]
[[ 0.06222983 -0.01407664]]
[[0.09988322 0.01220516]]
[[0.13363728 0.03917195]]
[[0.16636156 0.06429584]]
[[0.19883764 0.08648178]]
900 13
[[-0.24486831  0.01381202]]
[[-0.14066131 -0.01522461]]
[[-0.05291276 -0.03205167]]
[[ 0.00851111 -0.03037298]]
[[ 0.05019439 -0.01455013]]
[[0.08233076 0.00839829]]
[[0.11195578 0.03343582]]
[[0.14134686 0.05760746]]
[[0.17126392 0.07958231]]
901 9
[[-0.2753725   0.00807559]]
[[-0.17313237 -0.0201042 ]]
[[-0.08281855 -0.03859977]]
[[-0.01620724 -0.03915092]]
[[ 0.03075472 -0.02466782]]
[[ 0.06723583 -0.00189512]]
[[0.09951444 0.02362924]]
[[0.13096377 0.04861386]]
[[0.16248219 0.07149133]]
[[0.19378497 0.09159908]]
902 10
[[-0.24427079  0.01100112]]
[[-0.14187783 -0.01749942]]
[[-0.0559753  -0.03395813]]
[[ 0.00466617 -0.03241817]]
[[ 0.04662075 -0.01714175]]
[[0.0793929  0.00525702]

[[ 0.00506338 -0.03610612]]
[[ 0.04433179 -0.02113922]]
[[0.07500987 0.00113329]]
[[0.10250203 0.02557791]]
[[0.12944023 0.04929535]]
[[0.15786822 0.07098688]]
[[0.1873395  0.09022179]]
935 10
[[-0.24093461  0.00479024]]
[[-0.13663381 -0.02604935]]
[[-0.04805262 -0.0422027 ]]
[[ 0.01494583 -0.03815472]]
[[ 0.05877057 -0.01963265]]
[[0.0925456  0.00547335]]
[[0.1235546  0.03182211]]
[[0.15421808 0.05673113]]
[[0.18518007 0.07885323]]
936 9
[[-0.23365705  0.00427137]]
[[-0.12905823 -0.02651075]]
[[-0.04124231 -0.04166881]]
[[ 0.02022166 -0.03656127]]
[[-0.03543878 -0.04280252]]
[[ 0.02757164 -0.03501589]]
[[ 0.07172395 -0.01375812]]
[[0.10683396 0.01287704]]
[[ 0.08788841 -0.0072687 ]]
[[0.12651359 0.02110389]]
937 10
[[-0.2227901  0.0092519]]
[[-0.1194334 -0.0215215]]
[[-0.03395261 -0.03689629]]
[[ 0.02583896 -0.03233185]]
[[ 0.06714137 -0.01399617]]
[[0.09993617 0.01058228]]
[[0.13002896 0.03633926]]
[[0.16030455 0.0606571 ]]
[[0.19020198 0.08226234]]
938 9
[[-0.24136746  0.01926456]]


[[0.11760803 0.03880662]]
[[0.14514345 0.06225067]]
[[0.17332496 0.08302741]]
966 9
[[-0.20760477  0.01192524]]
[[-0.11062819 -0.01865172]]
[[-0.20515922  0.00942008]]
[[-0.1078181  -0.02092746]]
[[-0.03050843 -0.03429798]]
[[ 0.02171855 -0.02816864]]
[[ 0.05767663 -0.00940997]]
[[0.0867441  0.01472978]]
[[0.11510642 0.03959505]]
[[0.1440762  0.06287974]]
[[0.17386925 0.08358619]]
967 11
[[-0.24114992  0.00690891]]
[[-0.14386424 -0.02316252]]
[[-0.06156379 -0.04014137]]
[[-0.00307236 -0.0380569 ]]
[[ 0.0370709  -0.02184561]]
[[0.06877285 0.00153384]]
[[0.09784272 0.02675853]]
[[0.12659693 0.05095701]]
[[0.1556605  0.07260386]]
[[0.18517564 0.09151299]]
968 10
[[-0.21785066  0.01376401]]
[[-0.11915622 -0.01765685]]
[[-0.03844669 -0.03322817]]
[[ 0.01688589 -0.02919463]]
[[ 0.05476886 -0.01178697]]
[[0.08465353 0.0116315 ]]
[[0.11314623 0.03627299]]
[[0.14166266 0.05953912]]
[[0.13688926 0.04591247]]
969 9
[[-0.23480913  0.01461863]]
[[-0.13393095 -0.01817189]]
[[-0.04704177 -0.03670986]